In [1]:
import chromadb
client = chromadb.PersistentClient()

In [2]:
client.delete_collection("answers")

In [3]:
answers = client.create_collection(
    name="answers"
)

In [4]:
import pandas as pd
from tqdm import tqdm

df = pd.read_excel("../resources/comfort_datasets.xlsx")
df

,user,answer,class
0,"가부장적인 사회, 오늘따라 더 짜증 나",가부장적인 사회는 남녀 모두에게 좋지 않죠. 남자에게는 과도한 책임감을 부여해서 지...,사연
1,"나 조금 힘들고, 우울해.",위로해 드리고 싶어요. 힘들지만 잘 견뎌줘서 고마워요.,사연
2,너만 그런거 아니야 그만 듣고 싶다,"밤이 저무는 것처럼 고민도, 아픔도 조금은 저물길 바라요. 오늘도 수고했어요. 다 ...",사연
3,"응원 좀 해줘 잘 될 거라고, 내가 반드시 해낼 거라고, 나 믿는다고",잘 될 거예요. 할 수 있어요. 비록 장애물이 있어 넘어지더라도 결국 해내실 거라는...,사연
4,쉬어도 괜찮아요.,네! 힘드실때 언제든 와서 쉬다 가요!!,사연
...,...,...,...
34472,TV를 틀어놓고 제대로 보는 것도 아니에요.,집중력이 부족하시군요. 그러면 공부할 때에도 다른 생각이 많이 들 거예요. 그리고 ...,증상
34473,TV를 틀어놓고 제대로 보는 것도 아니에요.,TV를 틀어놓고 집중해서 보지 않는 것은 낭비일 수 있겠죠. 하지만 그렇다고 너무 ...,증상
34474,Who can relate? 누가 알아주긴 할까?,알아주는 이가 없어서 많이 속상하신가요~? 분명히 선생님을 알아주는 이가 있어요! ...,사연
34475,WPW증후군 진단 받았는데 심각한 것 같진 않아서 따로 치료를 받은 적은 없어요.,그렇다면 더 큰 문제가 생기기 전에 전문가에게 상담을 받아보는 것이 좋을 것 같아요...,증상


In [6]:
from onnxruntime import InferenceSession
from transformers import AutoTokenizer
import torch
import numpy as np
from tqdm import tqdm

MODEL_PATH = "j5ng/et5-sentence-comfort"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
sess = InferenceSession("../models/onnx/encoder.onnx_uint8.onnx" , providers=["CPUExecutionProvider"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
def mean_pooling(model_output, attention_mask):
    model_output = torch.from_numpy(model_output[0])
    # First element of model_output contains all token embeddings
    token_embeddings = model_output
    attention_mask = torch.from_numpy(attention_mask)
    input_mask_expanded = attention_mask.unsqueeze(
        -1).expand(token_embeddings.size())
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask, input_mask_expanded, sum_mask

In [8]:
def embedding_query(query: str, normalize_embeddings=False) -> np.ndarray:
    # user turn sequence to query embedding
    model_inputs = tokenizer(query, return_tensors="pt")
    inputs_onnx = {k: v.cpu().detach().numpy()
                   for k, v in model_inputs.items()}
    sequence = sess.run(None, inputs_onnx)
    query_embedding = mean_pooling(
        sequence, inputs_onnx["attention_mask"])[0][0]

    if normalize_embeddings:
        query_embedding = query_embedding / \
            np.linalg.norm(query_embedding)

    return query_embedding.numpy()

In [9]:
ids = []
metadatas = []
embeddings = [embedding_query(sen, normalize_embeddings=True) for sen in tqdm(df['user'].tolist())]

for row in tqdm(df.iterrows()):
    index = row[0]
    query = row[1]['user']
    answer = row[1]['answer']
    types = row[1]['class']
    
    metadata = {
        "query": query,
        "answer": answer
    }
    ids.append(str(index))
    metadatas.append(metadata)
    
chunk_size = 1024  # 한 번에 처리할 chunk 크기 설정
total_chunks = len(embeddings) // chunk_size + 1  # 전체 데이터를 chunk 단위로 나눈 횟수
embeddings = [ e.tolist() for e in tqdm(embeddings)]  

for chunk_idx in tqdm(range(total_chunks)):
    start_idx = chunk_idx * chunk_size
    end_idx = (chunk_idx + 1) * chunk_size
    
    # chunk 단위로 데이터 자르기
    chunk_embeddings = embeddings[start_idx:end_idx]
    chunk_ids = ids[start_idx:end_idx]
    chunk_metadatas = metadatas[start_idx:end_idx]
    
    # chunk를 answers에 추가
    answers.add(embeddings=chunk_embeddings, ids=chunk_ids, metadatas=chunk_metadatas)

100%|█████████████████████████████████████| 34477/34477 [14:34<00:00, 39.44it/s]
34477it [00:00, 35619.09it/s]
100%|███████████████████████████████████████████| 34/34 [00:24<00:00,  1.39it/s]


In [44]:
query_embeddings = embedding_query("진짜 내 마음은 뭘까?", normalize_embeddings=True).tolist()
result = answers.query(
    query_embeddings=query_embeddings,
    n_results=3
)

result_df = pd.DataFrame(result['metadatas'][0])
result_df['distances'] = result['distances'][0]
result_df

,answer,query,distances
0,그동안 선생님이 편하다고 느끼는 사람을 만나지 못해서 그럴 수도 있어요. 정말 편하...,마음은 어떻게 여는거야?,0.468593
1,답답하셨겠어요. 지금부터 알아가면 되지 않을까요? 인생에서 가장 힘드시기는 자기 자...,나도 내 마음을 모르니까 답답해.,0.582045
2,좋아하는 사람의 행동이나 말투를 말씀해 주시면 제가 느끼는 바를 말씀해 드릴께요.,내가 누군가를 또는 누군가가 나를 좋아하는 마음을 어떻게 알 수 있을까?,0.624415
